# FRC Match Team Metrics
Displays the historical metrics of teams at an event.

## Setup
In your virtual environment, install pandas and matplotlib: 
  `pip install pandas matplotlib`
* If you are using VS Code, it should ask you to install the IPython extensions.
* If this next cell runs with no errors, you are all set.

In [1]:
import util
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

## Functions to get Team and Event names
Some parts of this notebook depend on a team you are interested in.  Other parts of this notebook need to look up the team name.  This section defines a function to look up the team name.  Same things for event name.

In [2]:
def get_team_name(team_id):
    url = f'https://www.thebluealliance.com/api/v3/team/{team_id}'
    return util.call_tba_api(url).json()['nickname']

def get_event_name(event_id):
    url = f'https://www.thebluealliance.com/api/v3/event/{EVENT_KEY}/simple'
    resp = util.call_tba_api(url).json()
    return str(resp['year']) + ' ' + resp['name']

## Default Team and Event

In [3]:
YEAR = '2024'
TEAM = 'frc6223'
EVENT_KEY = '2024wila'

TEAM_NAME = get_team_name(TEAM)
EVENT_NAME = get_event_name(EVENT_KEY)

print(TEAM_NAME, 'at', EVENT_NAME)

Arsenal of Engineering at 2024 Seven Rivers Regional


## Teams at the Event

In [4]:
url = f'https://www.thebluealliance.com/api/v3/event/{EVENT_KEY}/teams/simple'
resp = util.call_tba_api(url).json()
df = pd.DataFrame.from_dict(resp, orient='columns')
print('Team count: ', str(len(df)))
del url, resp
df.drop(columns=['country', 'name', 'team_number'], inplace=True)
df.head(5)


Team count:  54


,city,key,nickname,state_prov
0,Hartford,frc1091,Oriole Assault,Wisconsin
1,Pewaukee,frc1259,Paradigm Shift,Wisconsin
2,Middleton,frc1306,BadgerBOTS,Wisconsin
3,Winnebago,frc1625,Winnovation,Illinois
4,Milwaukee,frc1675,UPS (Ultimate Protection Squad),Wisconsin


## Loop through teams to get metrics

In [5]:
df2 = pd.DataFrame(columns=['key', 'events', 'qual_wins', 'qual_loss', 'qual_ties', 'poff_wins', 'poff_loss', 'poff_ties', 'ranking'])
for team in df['key']:
    print('Researching: ', team)
    
    matches = 0
    qual_wins = 0
    qual_loss = 0
    qual_ties = 0
    poff_wins = 0
    poff_loss = 0
    poff_ties = 0
    ranking = []
    
    
    url = f'https://www.thebluealliance.com/api/v3/team/{team}/events/{YEAR}/statuses'
    resp = util.call_tba_api(url).json()
    
    for match in resp:
        details = resp[match]
        
        if details != None and details['qual'] != None:
            matches += 1
            qual_wins += details['qual']['ranking']['record']['wins']
            qual_loss += details['qual']['ranking']['record']['losses']
            qual_ties += details['qual']['ranking']['record']['ties']
            if details['playoff'] != None:
                poff_wins += details['playoff']['record']['wins'] 
                poff_loss += details['playoff']['record']['losses'] 
                poff_ties += details['playoff']['record']['ties'] 
            ranking.append(details['qual']['ranking']['rank'])

    df2.loc[len(df2)] = [team, matches, qual_wins, qual_loss, qual_ties, poff_wins, poff_loss, poff_ties, 0 if len(ranking) == 0 else sum(ranking) / len(ranking)]
    print('  ', matches, qual_wins, qual_loss, qual_ties, poff_wins, poff_loss, poff_ties, ranking)
    
df = df.join(df2.set_index('key'), on='key')
del df2

Researching:  frc1091
   1 5 4 1 0 0 0 [23]
Researching:  frc1259
   1 4 5 1 0 2 0 [17]
Researching:  frc1306
   1 5 5 0 0 0 0 [33]
Researching:  frc1625
   1 7 2 0 1 2 0 [9]
Researching:  frc1675
   1 4 6 0 0 0 0 [28]
Researching:  frc171
   1 4 7 0 0 0 0 [26]
Researching:  frc1710
   1 10 2 0 2 2 0 [6]
Researching:  frc1716
   1 2 7 0 0 0 0 [47]
Researching:  frc2129
   1 6 3 0 3 2 0 [13]
Researching:  frc2194
   1 6 4 0 3 2 0 [13]
Researching:  frc2202
   1 6 4 0 0 0 0 [25]
Researching:  frc2506
   1 2 8 0 0 0 0 [42]
Researching:  frc2704
   1 6 5 0 1 2 0 [21]
Researching:  frc2826
   1 5 6 0 0 2 0 [19]
Researching:  frc2830
   1 4 6 0 3 3 0 [24]
Researching:  frc2977
   1 8 1 0 0 2 0 [3]
Researching:  frc3082
   1 4 5 0 1 2 0 [22]
Researching:  frc3090
   0 0 0 0 0 0 0 []
Researching:  frc3381
   1 6 3 1 0 2 0 [10]
Researching:  frc3418
   1 4 5 0 0 0 0 [33]
Researching:  frc3692
   1 3 7 0 0 0 0 [46]
Researching:  frc4011
   1 5 3 1 0 0 0 [22]
Researching:  frc4021
   1 1 8 0 0 0 

In [12]:
df.sort_values(by=['qual_wins', 'ranking'], ascending=[False, True], inplace=True, ignore_index=True)
df

,city,key,nickname,state_prov,events,qual_wins,qual_loss,qual_ties,poff_wins,poff_loss,poff_ties,ranking
0,Menomonee Falls,frc6223,Arsenal of Engineering,Wisconsin,1,10,0,0,1,2,0,1.0
1,Olathe,frc1710,The Ravonics Revolution,Kansas,1,10,2,0,2,2,0,6.0
2,La Crescent,frc2977,Sir Lancer Bots,Minnesota,1,8,1,0,0,2,0,3.0
3,Sterling,frc6651,Golden Bots,Illinois,1,8,3,0,0,2,0,9.0
4,Cedar Falls,frc525,Swartdogs,Iowa,1,7,2,0,2,2,0,6.0
5,Winnebago,frc1625,Winnovation,Illinois,1,7,2,0,1,2,0,9.0
6,Duluth,frc4230,TopperBots,Minnesota,1,6,2,1,6,1,0,3.0
7,Manitowoc,frc3381,Droid Rage,Wisconsin,1,6,3,1,0,2,0,10.0
8,Minneapolis,frc2129,Ultraviolet,Minnesota,1,6,3,0,3,2,0,13.0
9,Fond du Lac,frc2194,Fondy Fire,Wisconsin,1,6,4,0,3,2,0,13.0


In [11]:
print(df)

               city      key                                      nickname  \
35  Menomonee Falls  frc6223                        Arsenal of Engineering   
6            Olathe  frc1710                       The Ravonics Revolution   
15      La Crescent  frc2977                               Sir Lancer Bots   
36         Sterling  frc6651                                   Golden Bots   
27      Cedar Falls   frc525                                     Swartdogs   
3         Winnebago  frc1625                                   Winnovation   
24           Duluth  frc4230                                    TopperBots   
18        Manitowoc  frc3381                                    Droid Rage   
8       Minneapolis  frc2129                                   Ultraviolet   
9       Fond du Lac  frc2194                                    Fondy Fire   
12       Naperville  frc2704                              Roaring Robotics   
26       New Berlin  frc5148                              New Be

In [21]:
df[df['key'] == 'frc8531'].index[0]


36

In [26]:
url = f'https://www.thebluealliance.com/api/v3/event/{EVENT_KEY}/matches/simple'
resp = util.call_tba_api(url).json()
for match in resp:
    if match['match_number'] in [7, 15, 22, 30, 45, 54, 62, 68, 79, 89]:
        print('Match', match['match_number'])
        print('  Red:')
        for team in match['alliances']['red']['team_keys']:
            print('    ', team, df[df['key'] == team].index[0] + 1) 
        print('  Blue:')
        for team in match['alliances']['blue']['team_keys']:
            print('    ', team, df[df['key'] == team].index[0] + 1) 
    # print(match['key'], match['alliances']['red']['team_keys'], match['alliances']['blue']['team_keys'], match['winning_alliance'])

Match 15
  Red:
     frc6223 1
     frc525 5
     frc3082 25
  Blue:
     frc1675 28
     frc2202 13
     frc3418 30
Match 22
  Red:
     frc2506 41
     frc6223 1
     frc9578 54
  Blue:
     frc2194 10
     frc6732 51
     frc5019 49
Match 30
  Red:
     frc8531 37
     frc967 29
     frc6223 1
  Blue:
     frc5914 33
     frc4021 46
     frc8700 42
Match 45
  Red:
     frc3692 40
     frc7021 16
     frc4230 7
  Blue:
     frc6223 1
     frc3418 30
     frc7103 14
Match 54
  Red:
     frc8531 37
     frc1710 2
     frc2704 11
  Blue:
     frc171 27
     frc8782 52
     frc6223 1
Match 62
  Red:
     frc1625 6
     frc6223 1
     frc5019 49
  Blue:
     frc1091 19
     frc8803 20
     frc8782 52
Match 68
  Red:
     frc1091 19
     frc6223 1
     frc5903 23
  Blue:
     frc5339 38
     frc4011 18
     frc9578 54
Match 7
  Red:
     frc3090 47
     frc5826 39
     frc9149 53
  Blue:
     frc7619 36
     frc2129 9
     frc6223 1
Match 79
  Red:
     frc4054 48
     frc3381 8
     frc95